In [25]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import seaborn as sns
import matplotlib .pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn .svm import SVC
import joblib
from joblib import dump
import h5py
import json

In [26]:
file_path = 'D:\\Dataset\\#4_전력 설비 에너지 패턴 및 고장 분석 센서\\Training\\1.펌프_일반모터\\2.SOH진단\\1.역률평균\\0.Combined\\'
files = ['Combined_LabelledData_346_역률평균.json', 'Combined_LabelledData_348_역률평균.json']
x_label = []
y_label = []

for file_name in files:
    full_path = file_path + file_name
    with open(full_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for entry in data['data']:
        if entry['ITEM_NAME'] == "역률평균":
            x_label.append(entry['ITEM_VALUE'])
            # y_label.append(entry['LABEL_NAME'])
            # 정상 -> 0
            if entry['LABEL_NAME']=='정상':
                y_label.append(0)
            # 주의 -> 1
            elif entry['LABEL_NAME']=='주의':
                y_label.append(1)
            # 경고 -> 2
            else:
                y_label.append(2)

x_label = np.array(x_label)
x_label_scaled = (x_label - np.mean(x_label)) / np.std(x_label)

In [27]:
data_indices = np.arange(len(x_label))
print(data_indices)
np.random.shuffle(data_indices)
print(data_indices)

x_label = np.array(x_label)[data_indices]
y_label = np.array(y_label)[data_indices]

x_train, x_temp, y_train, y_temp = train_test_split(x_label, y_label, test_size=0.4, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)
x_train = x_train.reshape(-1,1)
x_test = x_test.reshape(-1,1)

print(f"Train size: {len(x_train)}")
print(f"Validation size: {len(x_valid)}")
print(f"Test size: {len(x_test)}")


[    0     1     2 ... 57530 57531 57532]
[ 4079 41308 40600 ... 48684 15595  8275]
Train size: 34519
Validation size: 11507
Test size: 11507


In [28]:
def train_and_save_model(x_train, y_train, model_save_path):
    # RandomForestClassifier 모델 초기화 및 학습
    model = RandomForestClassifier()
    model.fit(x_train, y_train)

    # 모델 저장
    joblib.dump(model, model_save_path)

In [29]:
model_save_path = 'C:\\Users\\user\\Desktop\\test\\Define\\Define_add\\rf_model.pkl'
train_and_save_model(x_train, y_train, model_save_path)

In [30]:
# 모델 로드
loaded_model = joblib.load('C:\\Users\\user\\Desktop\\test\\Define\\Define_add\\rf_model.pkl')  # 모델 파일 경로로 변경

# 테스트 데이터에 대한 예측 수행
y_pred = loaded_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Confusion matrix 출력
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

# Classification report 출력
classification_rep = classification_report(y_test, y_pred)
print('Classification Report:')
print(classification_rep)


Accuracy: 0.9966
Confusion Matrix:
[[2342    0    0]
 [   1 4185   23]
 [   0   15 4941]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2342
           1       1.00      0.99      1.00      4209
           2       1.00      1.00      1.00      4956

    accuracy                           1.00     11507
   macro avg       1.00      1.00      1.00     11507
weighted avg       1.00      1.00      1.00     11507

